# Natural Language Processing with RNNs

### Sequence Data

### Encoding Text

## Bag of Words

In [9]:
# map word to integer representing it
vocab = {}
word_encoding = 1

In [13]:
def bag_of_words(text):
    global word_encoding

    # create a list of all of the word
    words = text.lower().split(" ")
    # stores all of the encodings and their frequency
    bag = {}

    for word in words:
        if word in vocab:
            # get encoding from vocab
            encoding = vocab[word]
        else:
            vocab[word] = word_encoding
            word_encoding += 1

        if encoding in bag:
            bag[encoding] += 1
        else:
            bag[encoding] = 1
    
    return bag

In [15]:
text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


## Sentiment Analysis

### Movie Review Dataset

In [16]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf 
import os
import numpy as np 

In [17]:
VOCAB_SIZE = 88584
MAXLEN = 250
BATCH_SIZE = 64

In [18]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

### More Preprocessing

In [19]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

### Creating the Model

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Training

In [22]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])

In [23]:
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10


625/625 [==============================] - 39s 58ms/step - loss: 0.4498 - acc: 0.7808 - val_loss: 0.3831 - val_acc: 0.8514
Epoch 2/10
625/625 [==============================] - 32s 52ms/step - loss: 0.2580 - acc: 0.9004 - val_loss: 0.2815 - val_acc: 0.8864
Epoch 3/10
625/625 [==============================] - 32s 52ms/step - loss: 0.1980 - acc: 0.9270 - val_loss: 0.3736 - val_acc: 0.8750
Epoch 4/10
625/625 [==============================] - 33s 53ms/step - loss: 0.1605 - acc: 0.9448 - val_loss: 0.2872 - val_acc: 0.8842
Epoch 5/10
625/625 [==============================] - 33s 53ms/step - loss: 0.1352 - acc: 0.9531 - val_loss: 0.3332 - val_acc: 0.8838
Epoch 6/10
625/625 [==============================] - 35s 56ms/step - loss: 0.1125 - acc: 0.9629 - val_loss: 0.3674 - val_acc: 0.8810
Epoch 7/10
625/625 [==============================] - 31s 49ms/step - loss: 0.0936 - acc: 0.9696 - val_loss: 0.3567 - val_acc: 0.8850
Epoch 8/10
625/625 [==============================] - 34s 54

In [24]:
result = model.evaluate(test_data, test_labels)
print(result)

782/782 [==============================] - 12s 16ms/step - loss: 0.4726 - acc: 0.8608
[0.4726109802722931, 0.86080002784729]


### Making Predictions

In [28]:
from tensorflow import keras

In [29]:
word_index = imdb.get_word_index()

In [32]:
def encode_text(text):
    tokens = keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens], MAXLEN)[0]

In [33]:
text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [34]:
# while were at it lets a decode function
reverse_word_index = {value: key for (key, value) in word_index.items()}

In [37]:
def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
        if num != PAD:
            text += reverse_word_index[num] + " "
        
    return text[:-1]

In [38]:
print(decode_integers(encoded))

that movie was just amazing so amazing


In [47]:
# make a prediction
def predict(text):
    encoded_text = encode_text(text)
    pred = np.zeros((1, 250))
    pred[0] = encoded_text
    result = model.predict(pred)
    print(result[0])

In [48]:
positive_review = "That movie was so awesome! I really loved it and would watch it again because it was amazingly great"
predict(positive_review)

1/1 [==============================] - 1s 650ms/step
[0.8943222]


In [52]:
negative_review = "that movie sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

1/1 [==============================] - 0s 33ms/step
[0.22685647]


## RNN Play Generator

In [56]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

### Dataset

In [58]:
path_to_file = tf.keras.utils.get_file("shakespeare.txt", "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

1115394/1115394 [==============================] - 1s 1us/step


### Loading Own Data

In [ ]:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

### Read Contents of File

In [59]:
# read, then decode for py2 compat
text = open(path_to_file, "rb").read().decode(encoding="utf-8")

# length of text is the number of characters in it
print("Length of text: {} characters".format(len(text)))

Length of text: 1115394 characters


In [60]:
### first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Encoding

In [61]:
vocab = sorted(set(text))

# creating a mapping from unique characters to indices
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [62]:
def text_to_int(text):
    return np.array([char2idx[c] for c in text])

In [63]:
text_as_int = text_to_int(text)

In [64]:
# how part of our text is encoded
print("Text: ", text[:13])
print("Encoded: ", text_to_int(text[:13]))

Text:  First Citizen
Encoded:  [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [65]:
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return "".join(idx2char[ints])

In [66]:
print(int_to_text(text_as_int[:13]))

First Citizen


### Creating Training Examples

In [67]:
# length of sequence for a training example
seq_length = 100
examples_per_epoch = len(text) // (seq_length+1)

In [68]:
# create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [69]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [70]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [71]:
dataset = sequences.map(split_input_target)

In [72]:
for x, y in dataset.take(2):
    print("\n\nEXAMPLE\n")
    print("INPUT")
    print(int_to_text(x))
    print("\nOUTPUT")
    print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [73]:
BATCH_SIZE = 64
# vocab is number of unique characters
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024
# buffer size to shuffle the dataset
# TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory, 
# Instead, it maintains a buffer in which it shuffles elements.
BUFFER_SIZE = 10000

In [74]:
data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Building the Model

In [76]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer="glorot_uniform"),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [78]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_2 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Creating a Loss Function

In [80]:
for input_example_batch, target_example_batch in data.take(1):
    # ask model for a prediction on our first batch of training data
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [82]:
# the prediction is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions[0])

64
tf.Tensor(
[[-0.00931592 -0.00268863  0.01062842 ...  0.0154992   0.00932966
  -0.00160682]
 [-0.00425422 -0.00059597  0.01095393 ...  0.0221952   0.00658926
  -0.0034372 ]
 [-0.00948711 -0.00014567  0.01100174 ...  0.02243002  0.00426236
  -0.0041007 ]
 ...
 [-0.00021257  0.00157258  0.0132362  ...  0.00955123  0.01543389
  -0.00019209]
 [ 0.00708039  0.00103649  0.01067642 ...  0.01118428  0.01162604
  -0.00123282]
 [ 0.01053786 -0.00408195  0.01412772 ...  0.0094512   0.00699033
   0.00119845]], shape=(100, 65), dtype=float32)


In [84]:
# examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

# this is a 2d array of length 100, each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00931592 -0.00268863  0.01062842 ...  0.0154992   0.00932966
  -0.00160682]
 [-0.00425422 -0.00059597  0.01095393 ...  0.0221952   0.00658926
  -0.0034372 ]
 [-0.00948711 -0.00014567  0.01100174 ...  0.02243002  0.00426236
  -0.0041007 ]
 ...
 [-0.00021257  0.00157258  0.0132362  ...  0.00955123  0.01543389
  -0.00019209]
 [ 0.00708039  0.00103649  0.01067642 ...  0.01118428  0.01162604
  -0.00123282]
 [ 0.01053786 -0.00408195  0.01412772 ...  0.0094512   0.00699033
   0.00119845]], shape=(100, 65), dtype=float32)


In [86]:
# a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

# its 65 values representing the probability of each character occuring next

65
tf.Tensor(
[-9.3159219e-03 -2.6886256e-03  1.0628422e-02 -7.1298564e-04
  3.0778358e-03 -5.1899427e-03 -2.4528732e-03  2.6836926e-03
  6.1333016e-03 -8.0910586e-03 -5.4772384e-03 -5.3892252e-03
 -7.8792004e-03 -5.0856406e-04  2.9113153e-03  4.1922904e-04
  1.4595806e-03 -8.5873324e-03 -8.0247065e-03 -5.7862252e-03
  7.2533083e-03 -2.0340579e-03  2.9454394e-03  2.4055922e-03
 -4.3149577e-03  8.2649309e-03 -2.9980845e-03 -4.9899155e-03
  4.1082567e-03 -2.4163662e-03 -3.9016004e-03 -2.4290432e-03
 -8.2970597e-03  8.0897473e-05  1.4212692e-02 -1.2347613e-03
 -2.0645615e-03  1.0640031e-02 -2.2951285e-03  5.9195622e-03
  5.4743048e-03  3.2921140e-03  3.2425835e-04  1.0469848e-02
  6.9688726e-04 -5.6243897e-03  6.5230643e-03  5.1852567e-03
 -1.1978524e-03 -8.1989970e-03 -5.9228875e-03  6.8507353e-03
 -1.4947127e-03 -6.3539166e-03 -1.3106934e-02 -2.2907569e-03
  2.8492277e-03 -6.0464684e-03  7.5841090e-06  1.1155365e-02
 -7.1356702e-04  4.0797540e-03  1.5499196e-02  9.3296617e-03
 -1.606817

In [87]:
# If we want to determine the predicted charater we need to sample the output distribution (pick a value based on probability)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

# what the model predicted for training sequence 1
print(predicted_chars)

BNLd3Y, IaRf;;
z$eLNVthfcgBHzliEbphc3lPMYGGD$rftaHSsbIJq;ZNRaPG qokZ.xtUdOlMiTwVfrX
lTssTv.DQiQ?i! g


In [95]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

### Compiling the Model

In [96]:
model.compile(optimizer="adam", loss=loss)

### Creating Checkpoints

In [97]:
# directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"

# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

### Training

In [98]:
history = model.fit(data, epochs=2, callbacks=[checkpoint_callback])

Epoch 1/2
172/172 [==============================] - 324s 2s/step - loss: 2.5540
Epoch 2/2
172/172 [==============================] - 326s 2s/step - loss: 1.8646


### Loading the Model

In [99]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [100]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

### Generating Text

In [101]:
def generate_text(model, start_string):
    # evaluation step (generating text using the learned model)
    
    # number of characters to generate
    num_generate = 800

    # converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # empty string to stor our results
    text_generated = []

    # low temperatures results in more preedictable text
    # higher temperatures results in more surprisin text
    # experiment to find the best setting
    temperature = 1.0

    # here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        prediction = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return (start_string + "".join(text_generated))

In [102]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

romeop som.

KIGG HENWARD II:
Way ke dove Iblove at I see:
If you what he sate that I not been like Mirriar!

KA note on dine arther deed, not, our my manuesoug onjecen,
Haw speny.

KONCEY:
Ay, if hal seed my parour wither my 'sweak,
We habd he are stheid ag elore from bepone that sway me numble my and
He chroof thould I kend mat, be mens
Way I contigaling. What, it it is refe-boors?
And tell ivuns han enger we senserd that with our tresp
Palonst;
Broodh vifferf him her this? I will fear, ere the withers?

LADY CASPEY:

LENCETIO:
My loods, That, rush ictrare, pood good;
He commpite-it mestice my pidse one herour
Brase, sonciture us to the sier ond hir;
buiringm in what, itl us, And which a frys it, Supit
A marent epus him trorid:
But but alper thr beares spower slowsh,
havish thoubling poldom p
